In [9]:
import os, sys, shutil, subprocess, textwrap, pathlib

PROJ = "/workspace/rag_streamlit"
VENV = f"{PROJ}/venv_streamlit"
PY   = f"{VENV}/bin/python"

# 0) 프로젝트 폴더 존재 확인
pathlib.Path(PROJ).mkdir(parents=True, exist_ok=True)
os.chdir(PROJ)
print("CWD:", os.getcwd())

# 1) 기존 venv 삭제 (완전 초기화)
if pathlib.Path(VENV).exists():
    print(">> 기존 venv 삭제 중:", VENV)
    shutil.rmtree(VENV)

# 2) 새 venv 생성
print(">> 새 venv 생성:", VENV)
subprocess.check_call([sys.executable, "-m", "venv", VENV])

# 3) pip 최신화
subprocess.check_call([PY, "-m", "pip", "install", "-U", "pip", "setuptools", "wheel"])

# 4) PyTorch 설치 (CUDA 있는 A40용) — 실패시 일반 torch로 폴백
def safe_install_torch():
    # Runpod 이미지에 따라 CUDA 12 계열이 일반적. 먼저 CUDA12용 인덱스로 시도.
    try:
        subprocess.check_call([PY, "-m", "pip", "install", "-U",
                               "torch", "torchvision", "torchaudio",
                               "--index-url", "https://download.pytorch.org/whl/cu121"])
        return
    except subprocess.CalledProcessError:
        pass
    try:
        subprocess.check_call([PY, "-m", "pip", "install", "-U",
                               "torch", "torchvision", "torchaudio",
                               "--index-url", "https://download.pytorch.org/whl/cu124"])
        return
    except subprocess.CalledProcessError:
        pass
    # 마지막 폴백(느리지만 호환): PyPI 일반
    subprocess.check_call([PY, "-m", "pip", "install", "-U", "torch", "torchvision", "torchaudio"])

print(">> PyTorch 설치 시도…")
safe_install_torch()

# 5) 나머지 패키지 한 번에 설치
pkgs = [
    # RAG/임베딩/FAISS
    "faiss-cpu", "sentence-transformers",
    # 모델/튜닝
    "transformers>=4.44", "accelerate>=0.33", "peft>=0.13", "bitsandbytes", "einops",
    # 앱
    "gradio", "streamlit", "python-dotenv",
    # PDF
    "pymupdf", "pypdf",
    # OpenAI 임베딩
    "openai",
]
print(">> 패키지 설치:", " ".join(pkgs))
subprocess.check_call([PY, "-m", "pip", "install", "-U"] + pkgs)

print("\n✅ 설치 완료")
print("Python:", subprocess.check_output([PY, "-V"], text=True).strip())


CWD: /workspace/rag_streamlit
>> 기존 venv 삭제 중: /workspace/rag_streamlit/venv_streamlit
>> 새 venv 생성: /workspace/rag_streamlit/venv_streamlit
  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.8 MB/s eta 0:00:00
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
>> PyTorch 설치 시도…
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 335.1 MB/s  0:00:0100:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 565.2 MB/s  0:00:00
     ━━━

In [ ]:
import subprocess, json, textwrap

PY = "/workspace/rag_streamlit/venv_streamlit/bin/python"

code = r"""
import sys, torch

print("PY:", sys.executable)

# PyMuPDF(fitz)
try:
    import fitz
    print("PyMuPDF:", getattr(fitz, "__doc__", "").splitlines()[0])
except Exception as e:
    print("PyMuPDF import ERROR:", e)

# Torch
print("torch:", torch.__version__, "| cuda_avail:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("device_count:", torch.cuda.device_count(), "| name:", torch.cuda.get_device_name(0))

# Transformers/Accelerate/PEFT
import transformers, accelerate, peft
print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)
print("peft:", peft.__version__)
"""
print(subprocess.check_output([PY, "-c", code], text=True))


In [ ]:
# 위치 이동
import os
os.chdir("/workspace/rag_streamlit")
!./venv_streamlit/bin/python /workspace/rag_streamlit/final_app.py

* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://a67b7740855e9138c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
^C
Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://a67b7740855e9138c4.gradio.live
